In [ ]:
import os
import numpy as np
import anndata as ad
import pandas as pd
import scanpy as sc
import copy
import warnings
from datetime import datetime

sc.settings.n_jobs = 32
warnings.filterwarnings("ignore")

pwd = os.getcwd()

### Update 1 - 2024-02-13: Add real ages for donors over 90 years old, full APOE4 genotypes, and whether a donor was severely affected. Also releasing donor-specific h5ad objects.

In [ ]:
region = "DLPFC"
date = "2023-07-19"
adata = sc.read_h5ad(os.path.join(pwd, "output", region, "SEAAD_DLPFC_RNAseq_all-nuclei." + date + ".h5ad"))

In [ ]:
metadata = pd.read_excel(
    os.path.join(pwd, "input", region, "Supplementary Table 1.xlsx"),
    sheet_name="SEA-AD_Cohort_Metadata",
    index_col=0
)
updated_metadata = adata.obs.loc[:, ["Donor ID"]].merge(
    metadata.loc[:, ["Age at Death", "APOE Genotype", "Severely Affected Donor"]].fillna("N"),
    left_on="Donor ID",
    right_index=True,
    how="left"
)

adata.obs["Age at Death"] = updated_metadata["Age at Death"].copy()
adata.obs["APOE4 Status"] = updated_metadata["APOE Genotype"].copy()
adata.obs = adata.obs.rename(
    {"APOE4 Status": "APOE Genotype"},
    axis=1,
)
adata.obs["Severely Affected Donor"] = updated_metadata["Severely Affected Donor"].copy()

In [ ]:
adata.write(os.path.join(pwd, "output", region, "SEAAD_DLPFC_RNAseq_all-nuclei." + str(datetime.date(datetime.now())) + ".h5ad"), compression="gzip")
adata.obs.to_csv(os.path.join(pwd, "output", region, "SEAAD_DLPFC_RNAseq_all-nuclei_metadata." + str(datetime.date(datetime.now())) + ".csv"))

In [ ]:
region = "DLPFC"
date = "2023-07-19"
adata = sc.read_h5ad(os.path.join(pwd, "output", region, "SEAAD_DLPFC_RNAseq_final-nuclei." + date + ".h5ad"))

In [ ]:
metadata = pd.read_excel(
    os.path.join(pwd, "input", region, "Supplementary Table 1.xlsx"),
    sheet_name="SEA-AD_Cohort_Metadata",
    index_col=0
)
updated_metadata = adata.obs.loc[:, ["Donor ID"]].merge(
    metadata.loc[:, ["Age at Death", "APOE Genotype", "Severely Affected Donor"]].fillna("N"),
    left_on="Donor ID",
    right_index=True,
    how="left"
)

adata.obs["Age at Death"] = updated_metadata["Age at Death"].copy()
adata.obs["APOE4 Status"] = updated_metadata["APOE Genotype"].copy()
adata.obs = adata.obs.rename(
    {"APOE4 Status": "APOE Genotype"},
    axis=1,
)
adata.obs["Severely Affected Donor"] = updated_metadata["Severely Affected Donor"].copy()

In [ ]:
adata.write(os.path.join(pwd, "output", region, "SEAAD_DLPFC_RNAseq_final-nuclei." + str(datetime.date(datetime.now())) + ".h5ad"), compression="gzip")
adata.obs.to_csv(os.path.join(pwd, "output", region, "SEAAD_DLPFC_RNAseq_final-nuclei_metadata." + str(datetime.date(datetime.now())) + ".csv"))

In [ ]:
for i in adata.obs["Donor ID"].cat.categories:
    print(i)
    adata[adata.obs["Donor ID"] == i].write(os.path.join(pwd, "output", region, "donor_objects", i + "_SEAAD_DLPFC_RNAseq_final-nuclei." + str(datetime.date(datetime.now())) + ".h5ad"), compression="gzip")